In [ ]:
import qrcode
#import sqlite3

# Connect to the database
conn = sqlite3.connect('users.db')
c = conn.cursor()

# Create a table to store the QR codes and their corresponding credits
c.execute('''CREATE TABLE IF NOT EXISTS qr_codes
             (code TEXT PRIMARY KEY, credits INTEGER)''')

# Create a table to store the user accounts and their scores
c.execute('''CREATE TABLE IF NOT EXISTS users
             (id INTEGER PRIMARY KEY, name TEXT, score INTEGER)''')

# Function to generate a QR code and insert it into the database
def generate_qr_code(credits):
    # Generate a unique code for the QR code
    code = uuid.uuid4().hex
    
    # Generate the QR code image
    qr = qrcode.QRCode(version=1, box_size=10, border=5)
    qr.add_data(code)
    qr.make(fit=True)
    img = qr.make_image(fill_color='black', back_color='white')
    
    # Save the QR code image to a file
    img.save(f'{code}.png')
    
    # Insert the QR code and credits into the database
    c.execute("INSERT INTO qr_codes (code, credits) VALUES (?, ?)", (code, credits))
    conn.commit()
    
    return code

# Function to scan a QR code and update the user's score
def scan_qr_code(code, user_id):
    # Check if the QR code exists in the database
    c.execute("SELECT * FROM qr_codes WHERE code=?", (code,))
    result = c.fetchone()
    if result is None:
        print("Invalid QR code.")
        return False
    
    # Check if the QR code has already been used
    if result[1] == 0:
        print("QR code already used.")
        return False
    
    # Update the user's score
    c.execute("UPDATE users SET score=score+? WHERE id=?", (result[1], user_id))
    conn.commit()
    
    # Set the credits of the QR code to 0 to prevent it from being used again
    c.execute("UPDATE qr_codes SET credits=0 WHERE code=?", (code,))
    conn.commit()
    
    print(f"{result[1]} credits added to user {user_id}.")
    return True


In [ ]:
import qrcode
import pandas as pd

data = pd.read_csv("QR_No.csv")

last_qr = data['QR']
scanned = list(data['QR_no'])


def generate_qr_code(credits,last_qr):

    # Generate a unique code for the QR code
    code = uuid.uuid4().hex
    
    # Generate the QR code image
    qr = qrcode.QRCode(version=1, box_size=10, border=5)
    qr.add_data(code)
    qr.make(fit=True)
    img = qr.make_image(fill_color='black', back_color='white')
    
    # Save the QR code image to a file
    img.save(f'{code}.png')
    
    return code
